# Advanced Topics for Lecture 9

## RSOME for optimization modeling

### Introduction to Robust Stochastic Optimization Made Easy (RSOME)
Among many packages capable of formulating and solving optimization problems, RSOME might be the most convenient one because it is compatible with NumPy N-dimensional arrays. 

<img src="https://github.com/XiongPengNUS/dao_resources/blob/main/rsome_intro.png?raw=true" width=900 style="border: 2.5px dashed #ccc"></img>

The package is installed by the following command.

In [1]:
!pip install rsome

  Using cached rsome-1.1.3-py3-none-any.whl (64 kB)


The RSOME modeling tools are imported by the following code segment.

In [1]:
from rsome import ro
import pandas as pd
import numpy as np

The package was developed and maintained by staffs at the Department of Analytics and Operations, NUS Business School. You may find the source code at [RSOME GitHub](https://github.com/XiongPengNUS/rsome), and detailed User Guide, as well as many application examples at the official website: [RSOME in Python](https://xiongpengnus.github.io/rsome/). Steps of using RSOME to solve optimization problems are summarized below:
1. Create an RSOME model.
2. Define variables associated with the created model.
3. Define the objective and constraints of the created model.
4. Solve the model using the selected solver.

<div class="alert alert-block alert-success" style="text-height:1.2">
<b>Example 5:</b>  
    Solve the linear programming problem:
\begin{align}
\text{max}~&3x_1 + 4x_2   \\
\text{s.t.}~& 2.5x_1 +x_2 \leq 20 \\
& 5x_1 + 3x_2 \leq 30 \\
& x_1 + 2x_2 \leq 16. \\
\end{align}
</div>

##### Method 1:

In [2]:
model = ro.Model()

x1 = model.dvar()                   # Define a decision variable x1
x2 = model.dvar()                   # Define a decision variable x2

model.max(3*x1 + 4*x2)              # Maximize the objective function
model.st(2.5*x1 + x2 <= 20)         # Specify the 1st constraints
model.st(5*x1 + 3*x2 <= 30)         # Specify the 2nd constraints
model.st(x1 + 2*x2 <= 16)           # Specify the 3rd constraints

model.solve()                       # Solve the model

Being solved by the default LP solver...
Solution status: 0
Running time: 0.0017s


We may then retrieve the optimal objective value and the optimal solution using the `get()` method.

In [3]:
model.get()

33.714285714285715

In [4]:
x1.get(), x2.get()

(1.7142857142857144, 7.142857142857142)

The RSOME package also provides analyzing and debugging tools such as export the model formulation as Pandas data frames.

In [5]:
model.do_math()

Conic program object:
Number of variables:           3
Continuous/binaries/integers:  3/0/0
---------------------------------------------
Number of linear constraints:  4
Inequalities/equalities:       4/0
Number of coefficients:        9
---------------------------------------------
Number of SOC constraints:     0
---------------------------------------------
Number of ExpCone constraints: 0

In [6]:
model.do_math().show()

,x1,x2,x3,sense,constant
Obj,1.0,0.0,0.0,-,-
LC1,0.0,2.5,1.0,<=,20.0
LC2,0.0,5.0,3.0,<=,30.0
LC3,0.0,1.0,2.0,<=,16.0
LC4,-1.0,-3.0,-4.0,<=,-0.0
UB,inf,inf,inf,-,-
LB,-inf,-inf,-inf,-,-
Type,C,C,C,-,-


##### Method 2:

We can also define variables $x_1$ and $x_2$ as an array $\pmb{x}=(x_1, x_2)$.

In [7]:
model = ro.Model()

x = model.dvar(2)                   # Define a variable as a 2-element array

model.max(3*x[0] + 4*x[1])          # Maximize the objective function
model.st(2.5*x[0] + x[1] <= 20)     # Specify the 1st constraints
model.st(5*x[0] + 3*x[1] <= 30)     # Specify the 2nd constraints
model.st(x[0] + 2*x[1] <= 16)       # Specify the 3rd constraints

model.solve()                       # Solve the model by OR-Tools

Being solved by the default LP solver...
Solution status: 0
Running time: 0.0016s


The optimal value and solution are the same as ***Method1***.

In [8]:
model.get()

33.714285714285715

In [9]:
x.get()

array([1.71428571, 7.14285714])

##### Method 3:

A third method is to define all model parameters as arrays, and the linear programming problem is specified by NumPy-style array operations. 

In [10]:
c = np.array([3, 4])            # Coefficient of the objective function
b = np.array([20, 30, 16])      # Right-hand-side coefficients of constraints
A = np.array([[2.5, 1],
              [5, 3],
              [1, 2]])          # Left-hand-side coefficients of constraints

In [11]:
model = ro.Model()

x = model.dvar(2)

model.max((c*x).sum())
model.st((A*x).sum(axis=1) <= b)

model.solve()

Being solved by the default LP solver...
Solution status: 0
Running time: 0.0012s


The optimal value and solution are given below.

In [12]:
model.get()

33.714285714285715

In [13]:
x.get()

array([1.71428571, 7.14285714])

### Application cases 

<div class="alert alert-block alert-success" style="text-height=1.2">
<b>Example 1:</b>  
The following table provides the cost and nutrition values of four different types of food. Find a minimum-cost diet that contains at least 1) 500 calories; 2) 6 grams of chocolate; 3) 10 grams of sugar; and 4) 8 grams of fat. 
</div>

In [14]:
diet_dict = {'Brownies': [400, 3, 2, 2, 0.5],
             'Ice Cream': [200, 2, 2, 4, 0.2], 
             'Cola': [150, 0, 4, 1, 0.3],
             'Cheese Cake': [500, 0, 4, 5, 0.8]}
diet = pd.DataFrame(diet_dict, 
                    index=['Calories', 'Chocolate', 'Sugar', 'Fat', 'Cost'])
diet

,Brownies,Ice Cream,Cola,Cheese Cake
Calories,400.0,200.0,150.0,500.0
Chocolate,3.0,2.0,0.0,0.0
Sugar,2.0,2.0,4.0,4.0
Fat,2.0,4.0,1.0,5.0
Cost,0.5,0.2,0.3,0.8


Coefficients given in the data frame above are extract as an array `A`. 

In [15]:
A = diet.values             
A

array([[4.0e+02, 2.0e+02, 1.5e+02, 5.0e+02],
       [3.0e+00, 2.0e+00, 0.0e+00, 0.0e+00],
       [2.0e+00, 2.0e+00, 4.0e+00, 4.0e+00],
       [2.0e+00, 4.0e+00, 1.0e+00, 5.0e+00],
       [5.0e-01, 2.0e-01, 3.0e-01, 8.0e-01]])

The nutrition requirements are defined as a one-dimensional array `b`. 

In [16]:
b = np.array([500, 6, 10, 8])
b

array([500,   6,  10,   8])

The diet problem can be formulated as
\begin{align}
\min~&A_{41}x_1 + A_{42}x_2 + A_{43}x_3 + A_{44}x_4 \\
\text{s.t.}~&A_{01}x_1 + A_{02}x_2 + A_{03}x_3 + A_{04}x_4 \geq b_0 \\
&A_{11}x_1 + A_{12}x_2 + A_{13}x_3 + A_{14}x_4 \geq b_1 \\
&A_{21}x_1 + A_{22}x_2 + A_{23}x_3 + A_{24}x_4 \geq b_2 \\
&A_{31}x_1 + A_{32}x_2 + A_{33}x_3 + A_{34}x_4 \geq b_3 \\
&x_1, x_2, x_3, x_4 \geq 0.
\end{align}
where $x_i$ is the quantity of the $i$th food. Such a linear programming problem can be solved by the following code. 

In [17]:
model = ro.Model()

x = model.dvar(4)                   # Define decision variables

model.min((A[-1, :]*x).sum())
model.st((A[:-1, :]*x).sum(axis=1) >= b)
model.st(x >= 0)
model.solve()

Being solved by the default LP solver...
Solution status: 0
Running time: 0.0033s


The optimal solution is presented below.

In [18]:
pd.Series(x.get(), index=diet.columns)

Brownies       0.0
Ice Cream      3.0
Cola           1.0
Cheese Cake    0.0
dtype: float64

<div class="alert alert-block alert-success" style="text-height=1.2">
<b>Example 2:</b>  
A furniture company is producing four types furniture: chairs, tables, stools, and shelves. All furniture products involves three operations: plaining, assembling, and painting. The profits of each type of furniture, and the required working minutes for each operation is presented in the table <span style='font-family:Courier'><b>furniture</b></span>. It is assumed that the due to a license issue, this company can at most produce 1200 shelf per month, and the maximum working minutes for each operation is given in the series <span style='font-family:Courier'><b>cap</b></span>. 
    <li>Write the formula of the linear programming model which maximizes the total profit.</li>
    <li>Use RSOME to implement the model.</li>
</div>

In [19]:
furniture = pd.DataFrame([[12.0, 18.0, 16.0, 30.0],
                          [5, 10, 12, 12],
                          [6, 3, 4, 5],
                          [5, 6.25, 5, 12.5]], 
                         index=['Profit', 'Plaining', 'Assembling', 'Painting'], 
                         columns=['Chair', 'Table', 'Stool', 'Shelf'])
furniture

,Chair,Table,Stool,Shelf
Profit,12.0,18.00,16.0,30.0
Plaining,5.0,10.00,12.0,12.0
Assembling,6.0,3.00,4.0,5.0
Painting,5.0,6.25,5.0,12.5


In [20]:
cap = pd.Series([27000.0, 18000.0, 25000.0], 
                index=['Plaining', 'Assembling', 'Painting'])
cap

Plaining      27000.0
Assembling    18000.0
Painting      25000.0
dtype: float64

- Let the array $\pmb{x}$ denote the production quantity of each type of furniture, the model formulation is written as:
$$
\begin{align}
\max ~ &\sum\limits_{i=1}^3A_{0i}x_i \\
\text{s.t.}~&\sum\limits_{i=1}^3A_{(j+1)i}x_i \leq c_j, &&j=0, 1, 2 \\
&x_3 \leq 1200 \\
&x_i \geq 0, &&i=0, 1, 2, 3
\end{align}
$$

where $A_{ji}$ is each coefficient in the `furniture` array, and $c_j$ is the element in the `cap` array.

The optimization model above can be implemented using RSOME.

In [21]:
A = furniture.values
c = cap.values

model = ro.Model()

x = model.dvar(4, vtype='I')

model.max((A[0]*x).sum())
model.st((A[1:]*x).sum(axis=1) <= c)
model.st(x[-1] <= 1200)
model.st(x >= 0)

model.solve()

Being solved by the default MILP solver...
Solution status: 0
Running time: 0.0089s


The maximum profit and the optimal decision are shown below.

In [22]:
model.get()

62076.0

In [23]:
pd.Series(x.get(), index=furniture.columns)

Chair    1136.0
Table     693.0
Stool       0.0
Shelf    1199.0
dtype: float64

<div class="alert alert-block alert-success" style="text-height=1.2">
<b>Example 3:</b>  
    Consider the furniture company in <b>Example 6</b>, if the company is paying workers extra for overtime, and the per minute rates are given in the series <span style='font-family:Courier'><b>overtime</b></span>. Update the linear programming model above and use RSOME to solve it.
</div>

In [24]:
overtime = pd.Series([2.0, 1.8, 1.5], 
                     index=['Plaining', 'Assembling', 'Painting'])
overtime

Plaining      2.0
Assembling    1.8
Painting      1.5
dtype: float64

Let $y_j$ be the amount of overtime spent on each step $j$, and $t_j$ be the corresponding cost for overtime, the updated model formulation is:

$$
\begin{align}
\max ~ &\sum\limits_{i=1}^3A_{0i}x_i - \sum\limits_{j=0}^2t_jy_j\\
\text{s.t.}~&\sum\limits_{i=1}^3A_{(j+1)i}x_i \leq c_j + y_j, &&j=0, 1, 2 \\
&x_3 \leq 1200 \\
&x_i \geq 0, &&i=0, 1, 2, 3 \\
&y_j \geq 0, &&j=0, 1, 2
\end{align}
$$

The updated version is implemented by the following code.

In [25]:
A = furniture.values
c = cap.values
t = overtime.values

model = ro.Model()

x = model.dvar(4, vtype='I')
y = model.dvar(3)

model.max((A[0]*x).sum() -  (t*y).sum())
model.st((A[1:]*x).sum(axis=1) <= c + y)
model.st(x[-1] <= 1200)
model.st(x >= 0)
model.st(y >= 0)

model.solve()

Being solved by the default MILP solver...
Solution status: 0
Running time: 0.0204s


When overtime is allowed, the maximum profit is increased.

In [26]:
model.get()

62209.875

The optimal solution in terms of the production quantities is presented below.

In [27]:
pd.Series(x.get(), index=furniture.columns)

Chair    1826.0
Table     347.0
Stool       0.0
Shelf    1200.0
dtype: float64

The decision on overtime minutes $\pmb{y}$ is given as follows.

In [28]:
pd.Series(y.get(), index=overtime.index)

Plaining         0.00
Assembling       0.00
Painting      1298.75
dtype: float64